In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install faiss-cpu
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 92 kB/s 


In [14]:
import faiss
import pickle

def exec_knn(data):
    # D = 768 # 768
    # K = 10000 # The number of clusters
    D = 768
    K = 10 # The number of clusters
    iter = 100

    # Kmeans Clutering
    kmeans = faiss.Kmeans(d=D, k=K, niter=iter, verbose=True, gpu=True)
    kmeans.train(data)

    cluster_centers_ = kmeans.centroids
    labels_ = kmeans.index.search(x=data, k=1)[1].reshape(-1)

    ## center 에서 가장 가까운 100개의 index, distance 추출
    index = faiss.IndexFlatL2(D)
    index.add(data)
    D, I = index.search(cluster_centers_.astype(np.float32), 100)

    # save
    save_path = '/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/pickles/text_embedding'
    os.makedirs(save_path, exist_ok=True)
    
    print('Saving Labels ...')
    with open(os.path.join(save_path, 'labels_k={}_iter={}.pickle'.format(K, iter)), 'wb') as f:
        pickle.dump(labels_, f, pickle.HIGHEST_PROTOCOL)
    
    print('Saving Cluster Centers ...')
    with open(os.path.join(save_path, 'cluster_centers_k={}_iter={}.pickle'.format(K, iter)), 'wb') as f:
        pickle.dump(cluster_centers_, f, pickle.HIGHEST_PROTOCOL)

    print('Saving Distance ...')
    with open(os.path.join(save_path, 'D_extract_data_k={}_iter={}_extract=100.pickle'.format(K, iter)), 'wb') as f:
        pickle.dump(D, f, pickle.HIGHEST_PROTOCOL)
    
    print('Saving Index ...')
    with open(os.path.join(save_path, 'I_extract_data_k={}_iter={}_extract=100.pickle'.format(K, iter)), 'wb') as f:
        pickle.dump(I, f, pickle.HIGHEST_PROTOCOL)

data = get_npy()
exec_knn(data)

Saving Labels ...
Saving Cluster Centers ...
Saving Distance ...
Saving Index ...


In [16]:
# load
with open('/content/drive/MyDrive/Colab Notebooks/Statistical_Data_Analysis/pickles/text_embedding/labels_k=10_iter=100.pickle', 'rb') as f:
    data = pickle.load(f)

print(data)

[5 7 5 0 5 3 1 0 0 6 1 9 7 2 9 9 6 8 9 9 0 0 5 9 0 3 7 1 9 5 8 7 7 3 9 7 7
 7 1 9 6 7 0 6 2 1 4 3 7 4 5 1 7 6 0 4 7 7 1 1 1 1 3 1 8 9 3 4 7 1 8 5 2 4
 9 0 1 3 9 7 9 9 0 2 2 7 4 8 7 1 8 4 8 5 0 1 1 4 7 1 3 0 6 1 8 3 0 7 9 8 8
 3 7 7 1 6 6 0 7 7 9 4 4 4 3 7 4 5 9 8 6 4 0 9 4 3 0 4 1 6 6 3 3 0 1 8 1 1
 0 7 1 7 0 7 9 5 1 6 5 5 9 9 8 1 4 1 6 3 5 4 7 7 1 3 7 4 4 7 9 7 8 0 1 4 9
 6 0 1 0 3 0 1 7 1 8 1 9 5 1 4 4 7 9 9 0 9 4 1 8 4 7 5 4 3 7 0 1 9 3 7 0 5
 0 4 9 9 7 8 3 7 7 8 8 1 0 3 8 4 5 0 1 4 4 3 1 1 7 9 7 3 4 9 4 1 4 5 1 5 6
 8 0 0 2 1 9 1 9 8 7 1 6 9 7 7 3 3 8 4 0 2 7 2 5 1 5 4 9 7 0 9 6 0 2 9 7 3
 5 0 7 1 3 7 7 4 7 7 3 0 1 9 8 7 5 4 3 7 4 7 7 3 2 7 8 3 7 1 7 7 8 8 2 1 5
 5 5 4 3 1 9 6 5 8 1 6 1 2 3 1 3 1 7 6 1 7 1 0 7 3 2 0 7 1 0 6 3]
